What I'm Trying top predict for this project is a number of rings on the shell of an Abalone ( A type of mollusk ) they are also called sea ears.
The number of rings is supposed to represent the age.

![Abalone](https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/LivingAbalone.JPG/220px-LivingAbalone.JPG)

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error

In [31]:
train_file = "./train.csv"
test_file = "./test.csv"

train = pd.read_csv(train_file, index_col=0)
test = pd.read_csv(test_file, index_col=0)
print(train.head())
print(test.head())

   Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
id                                                               
0    F   0.550     0.430   0.150        0.7715          0.3285   
1    F   0.630     0.490   0.145        1.1300          0.4580   
2    I   0.160     0.110   0.025        0.0210          0.0055   
3    M   0.595     0.475   0.150        0.9145          0.3755   
4    I   0.555     0.425   0.130        0.7820          0.3695   

    Whole weight.2  Shell weight  Rings  
id                                       
0           0.1465        0.2400     11  
1           0.2765        0.3200     11  
2           0.0030        0.0050      6  
3           0.2055        0.2500     10  
4           0.1600        0.1975      9  
      Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
id                                                                  
90615   M   0.645     0.475   0.155        1.2380          0.6185   
90616   M   0.580     0.460   0.160 

In [32]:
encoder = LabelEncoder()

train['Sex'] = encoder.fit_transform(train['Sex'])
test['Sex'] = encoder.transform(test['Sex'])
print(train['Sex'].head())

id
0    0
1    0
2    1
3    2
4    1
Name: Sex, dtype: int32


In [33]:
X = train.drop('Rings', axis=1)
y = train['Rings']

scaler = StandardScaler()
X = scaler.fit_transform(X)
test1 = scaler.transform(test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [34]:
linreg = LinearRegression()

linreg.fit(X_train, y_train)

y_pred = np.round(linreg.predict(X_test))
y_pred[y_pred<1] = 1

In [35]:
rmsle = root_mean_squared_log_error(y_true=y_test, y_pred=y_pred)
print(rmsle)

0.16745761106008042


That's a good baseline i think. would like to get it to at least 0.14 so i can get on the leaderboard. This will be my baseline.

## Model Iteration 4

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [20]:
# Function used to compile all the models
def compile_model(model, lr):
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.MeanSquaredLogarithmicError(
        reduction="sum_over_batch_size", name="mean_squared_logarithmic_error"
    )
    
    model.compile(optimizer=opt,
                  loss=loss,
                  metrics=['accuracy'])
    return model

In [21]:
def build_mlp(print_summary=False):
    model = Sequential()

    model.add(Input(shape=(X.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="relu"))

    if print_summary:
        print(model.summary())
    return model

mlp_4 = build_mlp(True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
compile_model(mlp_4, 0.001)

In [23]:
mlp_4.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
1416/1416 [==============================] - 5s 2ms/step - loss: 0.1609 - accuracy: 2.7589e-04
Epoch 2/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0261 - accuracy: 2.7589e-04
Epoch 3/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0245 - accuracy: 2.7589e-04
Epoch 4/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0241 - accuracy: 2.7589e-04
Epoch 5/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0239 - accuracy: 2.7589e-04
Epoch 6/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0238 - accuracy: 2.7589e-04
Epoch 7/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0237 - accuracy: 2.7589e-04
Epoch 8/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0236 - accuracy: 2.7589e-04
Epoch 9/10
1416/1416 [==============================] - 3s 2ms/step - loss: 0.0235 - accuracy: 2.7589e-04
Epoch 10/10
1416/1416 [=======================

In [26]:
y_pred_final_4 = np.round(mlp_4.predict(test1, batch_size=64))
y_pred_final_4[y_pred_final_4 < 1] = 1

944/944 [==============================] - 1s 965us/step


In [27]:
df_final_4 = pd.DataFrame()
df_final_4.index = test.index
df_final_4['Rings'] = y_pred_final_4

print(df_final_4.head())

       Rings
id          
90615    9.0
90616   10.0
90617   10.0
90618   10.0
90619    8.0


In [28]:
df_final_4['Rings'] = df_final_4['Rings'].astype('int')
print(df_final_4.head())

       Rings
id          
90615      9
90616     10
90617     10
90618     10
90619      8


In [29]:
df_final_4.to_csv('submission_4.csv')